In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

visited_urls = set()
emails_found = set()

def extract_emails_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find emails using regex
        new_emails = set(re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", response.text))
        emails_found.update(new_emails)

        # Find and crawl all links on the page
        for link in soup.find_all('a', href=True):
            full_url = urljoin(url, link['href'])
            if full_url not in visited_urls and urlparse(full_url).netloc == urlparse(url).netloc:
                visited_urls.add(full_url)
                extract_emails_from_url(full_url)

    except Exception as e:
        print(f"[Error] Failed to crawl {url} because: {e}")

def main():
    start_url = input("Enter a website URL to scrape for emails (e.g. https://example.com): ").strip()
    if not start_url.startswith('http'):
        start_url = 'http://' + start_url

    print(f"[*] Starting to crawl: {start_url}")
    visited_urls.add(start_url)
    extract_emails_from_url(start_url)

    # Save emails to file
    with open('emails_found.txt', 'w') as f:
        for email in sorted(emails_found):
            f.write(email + '\n')

    print(f"[✓] Found {len(emails_found)} email(s). Saved to emails_found.txt")

if __name__ == "__main__":
    main()



Enter a website URL to scrape for emails (e.g. https://example.com): smayrakhera@gmail.com
[*] Starting to crawl: http://smayrakhera@gmail.com
[✓] Found 0 email(s). Saved to emails_found.txt
